In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import util
%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


In [ ]:
possibleConvFilters1 = util.createRangeFromMidpoint(13,26)
possibleConvFilters2 = util.createRangeFromMidpoint(46,92)
possibleConvFilters3 = util.createRangeFromMidpoint(17,34)
possibleConvFilters4 = util.createRangeFromMidpoint(16,32)

possibleNumberOfFCLayers = util.createRangeFromMidpoint(27,54)
possibleNumberOfNeuronsPerFCLayer = util.createRangeFromMidpoint(33,66)

possibleNumberOfEpochs = util.createRangeFromMidpoint(22,44)
dropoutCriticalPoints = (0,1)
adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3)   

trial = 0

train = util.loadData("training")
dev,test = util.loadData("testing",.5)

choose = np.random.choice

n_convFilters1 = []
n_convFilters2 = []
n_convFilters3 = []
n_convFilters4 = []
n_FCLayers = []
n_NeuronsPerFCLayers = []
n_Epochs = []
dropoutRates = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

while trial < 50:
    convFilters1 = choose(possibleConvFilters1)
    convFilters2 = choose(possibleConvFilters2)
    convFilters3 = choose(possibleConvFilters3)
    convFilters4 = choose(possibleConvFilters4)

    numberOfFCLayers = choose(possibleNumberOfFCLayers)
    numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

    numberOfEpochs = choose(possibleNumberOfEpochs)
    
    dropoutRate = util.generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
    adamLearningRate = util.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = util.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    

    model = util.createModel(convFilters1, convFilters2, convFilters3, convFilters4,
                        numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate,L2Rate)

    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        print()
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if devScore > 0.91:
            model_path = f'../Models/fd_model_{trial}.h5'
            model.save(model_path)

        n_convFilters1.append(convFilters1)
        n_convFilters2.append(convFilters2)
        n_convFilters3.append(convFilters3)
        n_convFilters4.append(convFilters4)
        
        n_FCLayers.append(numberOfFCLayers)
        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        dropoutRates.append(dropoutRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = util.createModelParametersDF([convFilters1],[convFilters2],[convFilters3],[convFilters4],
                                              [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                              [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
        display(failedTrial)
    
        
    if (trial % 10 == 9): 
        modelParametersDF = util.createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                                                n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                                                dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleConvFilters1 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters1']),6*np.std(top5['n_convFilters1']))
        possibleConvFilters2 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters2']),6*np.std(top5['n_convFilters2']))
        possibleConvFilters3 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters3']),6*np.std(top5['n_convFilters3']))
        possibleConvFilters4 = util.createRangeFromMidpoint(np.mean(top5['n_convFilters4']),6*np.std(top5['n_convFilters4']))
    
        possibleNumberOfFCLayers = util.createRangeFromMidpoint(np.mean(top5['n_FCLayers']),6*np.std(top5['n_FCLayers']))
        possibleNumberOfNeuronsPerFCLayer = util.createRangeFromMidpoint(np.mean(top5['n_NeuronsPerFCLayers']),6*np.std(top5['n_NeuronsPerFCLayers']))
    
        possibleNumberOfEpochs = util.createRangeFromMidpoint(np.mean(top5['n_Epochs']),6*np.std(top5['n_Epochs']))
        dropoutCriticalPoints = util.calculateCriticalPoints(top5['dropoutRate'])
        adamLearningRateCriticalPoints = util.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = util.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_convFilters1 = []
        n_convFilters2 = []
        n_convFilters3 = []
        n_convFilters4 = []
        n_FCLayers = []
        n_NeuronsPerFCLayers = []
        n_Epochs = []
        dropoutRates = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []
            
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-07-09 06:59:34.037623: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-09 06:59:34.037799: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-09 06:59:35.108946: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 8s 101ms/step - loss: 0.3514 - f1_score: 0.6487
devScore


2024-07-09 07:06:26.306910: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-07-09 07:06:26.307062: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 1s 62ms/step - loss: 0.3658 - f1_score: 0.6277
concluding trial  0

trainScore
75/75 [==============================] - 10s 123ms/step - loss: 0.3511 - f1_score: 0.6484
devScore
10/10 [==============================] - 1s 82ms/step - loss: 0.3692 - f1_score: 0.6244
concluding trial  1

trainScore
75/75 [==============================] - 7s 80ms/step - loss: 0.3507 - f1_score: 0.6489
devScore
10/10 [==============================] - 1s 39ms/step - loss: 0.3663 - f1_score: 0.6384
concluding trial  2

trainScore
75/75 [==============================] - 7s 84ms/step - loss: 0.3504 - f1_score: 0.6497
devScore
10/10 [==============================] - 1s 45ms/step - loss: 0.3743 - f1_score: 0.6196
concluding trial  3

trainScore
75/75 [==============================] - 6s 77ms/step - loss: 0.3506 - f1_score: 0.6497
devScore
10/10 [==============================] - 1s 37ms/step - loss: 0.3675 - f1_score: 0.6320
concluding trial  4

trainScore
75/75 [===

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
4,1,10,33,11,40,50,16,0.281805,0.004401,0.022946,0.649686,0.631977
3,4,64,11,30,7,32,43,0.143499,0.000136,0.035179,0.649674,0.619625
5,4,50,20,18,37,52,2,0.888817,0.000123,1.559917,0.649496,0.645058
2,1,31,5,2,48,9,41,0.359425,0.000861,0.833504,0.648945,0.638418
8,3,75,6,30,20,53,42,0.780058,0.006216,0.012160,0.648903,0.650836
7,8,42,26,21,45,45,12,0.050078,0.000207,0.527514,0.648878,0.637643
0,24,21,16,20,37,50,20,0.269395,0.001709,2.102684,0.648740,0.627677
1,15,68,29,12,36,13,33,0.188070,0.000219,4.412820,0.648395,0.624442
6,21,12,31,27,49,37,41,0.458366,0.000684,1.998171,0.646586,0.638469



trainScore
75/75 [==============================] - 9s 113ms/step - loss: 0.3505 - f1_score: 0.6493
devScore
10/10 [==============================] - 1s 69ms/step - loss: 0.3678 - f1_score: 0.6460
concluding trial  9

trainScore
75/75 [==============================] - 8s 102ms/step - loss: 0.3502 - f1_score: 0.6499
devScore
10/10 [==============================] - 1s 63ms/step - loss: 0.3641 - f1_score: 0.6352
concluding trial  10

trainScore
75/75 [==============================] - 7s 83ms/step - loss: 0.3517 - f1_score: 0.6476
devScore
10/10 [==============================] - 1s 45ms/step - loss: 0.3694 - f1_score: 0.6304
concluding trial  11

trainScore
75/75 [==============================] - 8s 97ms/step - loss: 0.3928 - f1_score: 0.6487
devScore
10/10 [==============================] - 1s 59ms/step - loss: 0.4069 - f1_score: 0.6336
concluding trial  12

trainScore
75/75 [==============================] - 7s 77ms/step - loss: 1.4310 - f1_score: 0.6483
devScore
10/10 [===========

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
8,5,91,17,40,8,15,16,0.876725,0.000252,0.000284,0.650388,0.638245
1,3,47,43,42,6,27,51,0.541507,0.052340,0.576541,0.649901,0.635222
0,5,82,2,29,35,80,48,0.000000,0.000014,0.003954,0.649274,0.645991
7,5,63,43,48,28,79,9,0.999000,0.037024,0.077866,0.648966,0.634663
5,2,101,43,38,42,38,58,0.890246,0.000362,0.004692,0.648931,0.633097
3,2,65,19,46,24,52,5,0.999000,0.015246,0.021768,0.648653,0.633578
9,4,31,35,47,21,72,75,0.321148,0.000610,0.009953,0.648423,0.640589
4,2,16,18,13,64,51,6,0.473896,0.002916,1.058930,0.648259,0.638501
2,4,56,20,41,24,10,44,0.693011,0.000090,4.893488,0.647606,0.630377
6,4,33,2,27,30,65,17,0.999000,0.000017,0.000648,0.647537,0.642772
